Ucitavanje biblioteka za ucitavanje podataka

In [ ]:
import pandas

import numpy as np
import matplotlib.pyplot as plt
import cv2

%matplotlib inline

Definisanje globalnih promenljivih za lakse menjanje

In [ ]:
PUTANJA = 'C:/Users/User/Desktop/AI'
KATEGORIJE = ['ZDRAV', 'VIRUS', 'BAKTERIJA']
IMG_SIZE = 400

Ucitavamo biblioteku za logovanje performansi modela, i definisemo mu ime

In [ ]:
from tensorflow.keras.callbacks import TensorBoard
import time

IME = "Model-Pluca-9x2-{}-v2_0".format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(IME))

Pravljenje niza labela (train_y) i punimo ga brojevnim reprezentacijama zadatih kategorija

In [ ]:
csv_fajl = pandas.read_csv(PUTANJA + r"\chest_xray_metadata.csv")  # promeniti na svoju lokaciju

# uklanjamo pusace zbog manjka slika (takodje je tako navedeno u zadatku)

train_y = np.ndarray(0)

for index, row in csv_fajl.iterrows():
    if row['Label'] != 'Normal':
        if row['Label_1_Virus_category'] == 'Virus':
            train_y = np.append(train_y, 1)
        elif row['Label_1_Virus_category'] == 'bacteria':
            train_y = np.append(train_y, 2)
        else:
            print("Neodgovarajuca labela na slici {}".format(row['X_ray_image_name']))
    else:
        train_y = np.append(train_y, 0)
        
trainy2 = np.ndarray(0)

for index, row in csv_fajl.iterrows():
    if row['Label'] != 'Normal':
        if row['Label_1_Virus_category'] == 'Virus':
            trainy2 = np.append(trainy2, 1)
        elif row['Label_1_Virus_category'] == 'bacteria':
            trainy2 = np.append(trainy2, 2)
        else:
            trainy2 = np.append(trainy2, -1)
    else:
        trainy2 = np.append(trainy2, 0)

Ponovo prolazimo kroz fajl, ali ovaj put da ucitamo slike u niz (training_data), zajedno sa labelama iz proslog niza (train_y)

In [ ]:
csv_fajl.head()
training_data = []

for index, row in csv_fajl.iterrows():
    if trainy2[index] != -1:
        img_name = row['X_ray_image_name']
        lokacija = PUTANJA + '/snimci/' + str(img_name)
        try:# za slucaj da neka slika ostecena 
            slika = cv2.imread(lokacija, cv2.IMREAD_GRAYSCALE)
            slika = cv2.resize(slika, dsize=(IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)
            training_data.append([slika, int(trainy2[index])])
        except:
            print("Ostecena slika " + str(img_name))

BITNO: Moramo da se postaramo da slike niju u redosledu

In [ ]:
import random

random.shuffle(training_data)

Delimo niz (training_data) u 2 niza (X i y), i menjamo oblik (shape) niza X da bi tensorflow mogao da radi sa njima

In [ ]:
for features, label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

Poslednji korak

# Pravljenje, pokretanje, pracenje i cuvanje modela

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

from tensorflow.keras.optimizers import RMSprop

dense_layers = [0]
layer_sizes = [32]
conv_layers = [1]

for epp in [20]:#[1, 2, 3, 5, 10]:
    for dense_layer in dense_layers:
        for layer_size in layer_sizes:
            for conv_layer in conv_layers:
                NAME = "{}-dense---{}-nodes---{}-conv---{}-v2_0".format(dense_layer, layer_size, conv_layer, int(time.time()))
                tensorboard = TensorBoard(log_dir='logs_test/{}'.format(IME))
                print(NAME)

                model = tf.keras.models.Sequential()

                model.add(Conv2D(layer_size, (3, 3), input_shape = X.shape[1:]))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

                for l in range(conv_layer-1):
                    model.add(Conv2D(layer_size, (3, 3)))
                    model.add(Activation('relu'))
                    model.add(MaxPooling2D(pool_size=(2, 2)))

                model.add(Flatten())

                for l in range(dense_layer):
                    model.add(Dense(layer_size))
                    model.add(Activation('relu'))


                model.add(Dense(3))

                model.compile(optimizer='adam',
                              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                              metrics=['accuracy']
                )
                model.fit(np.array(X), np.array(y), batch_size=9,  epochs=epp, validation_split=0.1, callbacks=[tensorboard])
                model.save('saved_model\\100ep' + str(epp))